# Récupération des données

In [ ]:
import pandas as pd
from math import isnan
import time 
import numpy as np

In [ ]:
df = pd.read_excel("PS_INFO.xlsx")

In [ ]:
Form = df['Série diplôme']

In [ ]:
def normalisation_bac(i,forme,epreuve):
    ns = []
    add = 0
    for e in range(len(df)):
        if df['Série diplôme'][e] == forme:
            if isnan(df[epreuve][e]) == False:
                ns.append(df[epreuve][e])
    ecart_type = np.std(ns)
    moyenne = np.mean(ns)
    normalise = (df[epreuve][i]-moyenne)/ecart_type
    return normalise

In [ ]:
normalisation_bac(1,"Scientifique","Note à l'épreuve de Mathématiques")

In [ ]:
matiere = ["Mathématiques","Langue vivante A","Langue vivante B","Français","Systemes d''information et numerique"]
trimestre = ["Trimestre 1","Trimestre 2","Trimestre 3"]

In [ ]:
def normalisation_trimestre(df,i,matiere,trimestre):
    Moy=df["Moyenne classe en "+matiere+" "+trimestre][i]
    Min=df["Moyenne plus basse en "+matiere+" "+trimestre][i]
    Max=df["Moyenne plus haute en "+matiere+" "+trimestre][i]
    Can=df["Moyenne candidat en "+matiere+" "+trimestre][i]
    if(isnan(Can)) : 
        return None
    if(isnan(Moy) or isnan(Min) or isnan(Max)) :
        return None
    if(Max==Min) : 
        note = 0
    else : 
        note=(Can-Moy)/(Max-Min)
    return note


In [ ]:
COEF_GENERO = pd.DataFrame(
    {
        'Mathématiques': [1,2,3,2,3,4,13,8,10], 
        'SCIENCES': [1,2,3,2,3,4,4,8,None], 
        "Systemes d''information et numerique": [1,1,1,2,2,2,6,9,None],   
        "Informatique et Sciences du numérique": [1,1,1,2,2,2,6,8,None], 
        'Langue vivante A': [1,1,1,1,1,1,4,8,None], 
        'Langue vivante B': [1,1,1,1,1,1,4,8,None],
        'Français': [1,1,1,1,1,1,9,8,None]
    },
    index = ['P1', 'P2', 'P3', 'T1', 'T2', 'T3', 'COEF', 'PLANCHE_GEN','PLANCHE_STI2D']
)

In [ ]:
COEF_GENERO['Mathématiques']['PLANCHE_GEN']

In [ ]:
def moyenne_mat(df,i,matiere):
    tri = ["Trimestre 1.1","Trimestre 2.1","Trimestre 3.1","Trimestre 1","Trimestre 2","Trimestre 3"]
    z = 0
    d = 0
    divis = 0
    for e in tri:
        mns = normalisation_trimestre(df,i,matiere,e)
        if mns != None:
            mnse = mns*COEF_GENERO[matiere][d]
            divis += COEF_GENERO[matiere][d]
            z += mnse
        d+=1
    if z != 0:
        res1 = z/divis
        return res1
    

In [ ]:
moyenne_mat(df,11,"Systemes d''information et numerique")

In [ ]:
FR_Oral = df["Note à l'épreuve de Oral de Français (épreuve anticipée)"]
FR_Ecrit = df["Note à l'épreuve de Ecrit de Français (épreuve anticipée)"]

In [ ]:
a = recupDBC(df,456)
a[0]


In [ ]:

start = time.time()
df = pd.read_excel("PS_INFO.xlsx")
for i in range(len(df)):
    np.seterr(all='ignore')
    recupDBC(df,i)
end = time.time()
elapsed = end - start

print(elapsed)

In [ ]:
# Es-ce que le candidats à son Bac de Français ?
def BAC_FR(i):
    if (isnan(FR_Oral[i]) and isnan(FR_Ecrit[i])):
        return "NON"
    else :
        return "OUI"

In [ ]:
def TRI_MANQUE(i,matiere):
    T1=df["Moyenne candidat en "+matiere+" Trimestre 1.1"][i]
    T2=df["Moyenne candidat en "+matiere+" Trimestre 2.1"][i]
    T3=df["Moyenne candidat en "+matiere+" Trimestre 3.1"][i]
    P1=df["Moyenne candidat en "+matiere+" Trimestre 1.1"][i]
    P2=df["Moyenne candidat en "+matiere+" Trimestre 2.1"][i]
    P3=df["Moyenne candidat en "+matiere+" Trimestre 3.1"][i]
    PREMIERE = True
    TERMINALE = True
    if (isnan(P1) == False and isnan(P2) == False) and isnan(P3) == False:
        PREMIERE = False
    else:
        PREMIERE = True
        
    if (isnan(T1) == False and isnan(T2) == False) and isnan(T3) == False:
        TERMINALE = False
    else:
        TERMINALE = True
        
    if TERMINALE and PREMIERE == False:
        return False
    else:
        return True
    

In [ ]:
def MAT_FR(i):
    P1=df["Moyenne candidat en Français Trimestre 1.1"][i]
    P2=df["Moyenne candidat en Français Trimestre 2.1"][i]
    P3=df["Moyenne candidat en Français Trimestre 3.1"][i]
    if (isnan(P1) == False and isnan(P2) == False) and isnan(P3) == False:
        return "OUI"
    else:
        return "NON"

In [ ]:
def recupDBC(df,i):
    #Création de matière qui stock les notes du candidat
    matiere = pd.DataFrame()
    #Création d'une DataFrame pour le Candidats i
    DBC ={
        'Validation':0,
        'Num_Can':df['ID'][i],
        'SEXE':df['Sexe'][i], 
        'ANGLAIS':'?',
        'FILIERE':'?',
        'FORMATION':'?',
        'ANNEE_BAC':'?',
        'Bourse': False, 
        'Nom_Eta' : df["Libellé établissement"][i], 
        'Com_Eta' : df["Commune établissement"][i], 
        'Depa_Eta' : df["Département établissement"][i],
        'COMMENTAIRE':'?'
    }

    NOTE_DBC = {
        "Num_Can":df['ID'][i],
        "Mathématiques":None,
        "Français":None,
        "Informatique et Sciences du numérique":None,
        'Sciences économiques et sociales':None,
        "Anglais":None
    }
    NOTE_STI2D = {
        "Num_Can":df['ID'][i],
        "Mathématiques":None,
        "Français":None,
        "Systemes d''information et numerique":None,
        "Anglais":None 
    }
    NOTE_SG = {
        "Num_Can":df['ID'][i],
        "Mathématiques Spé":None,
        "Français":None,
        "Science":None,
        "Informatique et Sciences du numérique":None,
        "Anglais":None
    }
    
    
    #Récupération des informations:
    #Quelle est sa formations scolaire précédent sa candidatures
    Form = df['Série diplôme'][i]
    
    
    #Prise en compte des notes de BAC de Français
    FR_Oral = df["Note à l'épreuve de Oral de Français (épreuve anticipée)"][i]
    FR_Ecrit = df["Note à l'épreuve de Ecrit de Français (épreuve anticipée)"][i]   
    
    
    #Le Candidats est-il boursier ?
    if(df["Boursier"][i] in ["Boursier de l'enseignement supérieur", 'Boursier du secondaire']) : 
        DBC['Bourse'] = True # Mise à jour de sa situation de bourse
        
    #Es-ce que le candidats est en formation Générale ou Technologique ?
    if df["Groupe"][i] == 7415:
        DBC['FILIERE']='Générale'
    else : 
        DBC['FILIERE']='Technologique'
    #Année de Terminale/BAC
    DBC["ANNEE_BAC"] = df["Année scolaire"][i]
        
    
    #On vas attrivuer des points:
    # -1 vaut une élémination
    #  0 sa candidatures sera prise en compte en manuel
    #  1 sa candidatures vas être 
    if(df['Candidature validée (O/N)'][i]=='Non') : 
        # Si non validé éléminé
        DBC['Validation']= -1    
        DBC['COMMENTAIRE'] = "N'a pas Validé sa candidature ParcourSup"
        return DBC, matiere
    # On recherche les DAEU Mathématiques.
    # Dans un premier temps nous n'avons que les noms en DAEU
    elif Form == "DAEU" : 
        # On recherche plus précisemment les Daeu-b qui sont les DAEU MAthématiques
        if df["Spécialité diplôme bac Pro et anciens bacs"][i] == "Daeu-b" :
            # Dossier manuel
            DBC['Validation']=0
            DBC['Autre_Form']="Daeu-b"
            DBC['COMMENTAIRE'] = "La candidat a un DAEAU-B, donc Scientifique"
            return DBC , matiere
        else:
            # Si DAEU-a, donc Littérature éléminé
            DBC['Validation']=-1
            DBC['COMMENTAIRE'] = "La candidat a un DAEAU-A, donc de Littérature"
            return DBC , matiere
            
            
    # On recherche les anciens Bac ou Bac Internationale
    elif(Form in ["Bac général de plus de 15 ans","Baccalauréat International"]) : 
        #Bac en cours mais pas classique => dossier à étudier à la main
        DBC['Validation']=0
        DBC['Autre_Form']="AUTRES"
        return DBC , matiere
    
    # On cherche les Bac Scientifiques, qui ont validé leur Bac de français
    elif(Form=="Scientifique"): 
        if BAC_FR(i) == "NON":
            DBC['Validation']= 0
            DBC['FORMATION'] = "Scientifique"
            DBC['COMMENTAIRE'] = "N'a pas ses deux notes de Bac de Français"
            return DBC , matiere
        else:
            DBC["Validation"]=1
            DBC['FORMATION'] = "Scientifique"
            #Mathématiques, les baccalauréat Scientifiques sont des "Anciens Baccalauréat", 
            #et ou on déjà passé leurs baccalauréat
            math = []
            if isnan(df["Note à l'épreuve de Mathématiques"][i]) == True:
                DBC['Validation']= 0
                DBC['COMMENTAIRE'] = "Aucune Notes de Mathématiques pour l'épreuve du BAC"
                return DBC, matiere
            else:
                math.append(normalisation_bac(i,"Scientifique","Note à l'épreuve de Mathématiques"))
                NOTE_DBC['Mathématiques']= math[0]

            #Français
            fre = float(normalisation_bac(i,"Scientifique","Note à l'épreuve de Ecrit de Français (épreuve anticipée)"))
            fro = float(normalisation_bac(i,"Scientifique","Note à l'épreuve de Oral de Français (épreuve anticipée)"))
            frt = (fre+fro)/2
            NOTE_DBC['Français']= frt

            #Spécialité pour le baccalauréat Scientifique, les notes de Mathématiques Spé , sont incluse avec la note de Math
            # On ne vas rechercher que les notes d'ISN
            SPE=df["Note à l'épreuve de Informatique et Sciences du numérique"][i]
            nsi = normalisation_bac(i,"Scientifique","Note à l'épreuve de Informatique et Sciences du numérique")
            if isnan(SPE) == False  :
                NSI = "OK"
                NOTE_DBC["Informatique et Sciences du numérique"]= nsi
            else:
                NSI = "NON"



            #Anglais
            LV1=df["LV1"][i]
            LV11=df["LV1.1"][i]
            LV12=df["LV1.2"][i]
            LV2=df["LV2"][i]
            LV21=df["LV2.1"][i]
            LV22=df["LV2.2"][i]
            Choix = ["Note à l'épreuve de Langue vivante 1","Note à l'épreuve de Langue vivante 2"]
            angl=None
            if((LV1=="Anglais" or LV11 =="Anglais") or LV12 =="Anglais"  ): 
                angl="Langue vivante 1"
                DBC["ANGLAIS"] = "LV1"
                agl = normalisation_bac(i,"Scientifique","Note à l'épreuve de Langue vivante 1")

            if((LV2=="Anglais" or LV21 =="Anglais") or LV22 =="Anglais"  ): 
                angl="Langue vivante 1"
                DBC["ANGLAIS"] = "LV2"
                agl = normalisation_bac(i,"Scientifique","Note à l'épreuve de Langue vivante 2")

            if(angl!=None) : 
                NOTE_DBC['Anglais']= agl 
            else:
                if NSI == "OK":
                    DBC['Validation']= 0
                    DBC['COMMENTAIRE'] = "Note de Français, MAthématiques et NSI OK. Aucune Note d'Anglais"
                else:
                    DBC['Validation']= 0
                    DBC['COMMENTAIRE'] = "Note de Français, MAthématiques OK. Aucune Note d'Anglais"
            return DBC , NOTE_DBC

    # Recherche des Bac ES
    elif(Form=="Economique et social" ): 
        if BAC_FR(i) == "NON":
            DBC['Validation']= 0
            DBC['FORMATION'] = "Economique et social"
            DBC['COMMENTAIRE'] = "N'a pas ses deux notes de Bac de Français"
            return DBC , matiere
        else:
            DBC["Validation"]=1
            DBC['FORMATION'] = "Economique et social"
            #Mathématiques, les baccalauréat Scientifiques sont des "Anciens Baccalauréat", 
            #et ou on déjà passé leurs baccalauréat
            if isnan(df["Note à l'épreuve de Mathématiques"][i]) == True:
                DBC['Validation']= 0
                DBC['COMMENTAIRE'] = "Aucune Notes de Mathématiques pour l'épreuve du BAC"
                return DBC, matiere
            else:
                math = [] 
                math.append(normalisation_bac(i,"Economique et social","Note à l'épreuve de Mathématiques"))
                NOTE_DBC['Mathématiques']= math[0]

            #Français
            fre = float(normalisation_bac(i,"Economique et social","Note à l'épreuve de Ecrit de Français (épreuve anticipée)"))
            fro = float(normalisation_bac(i,"Economique et social","Note à l'épreuve de Oral de Français (épreuve anticipée)"))
            frt = (fre+fro)/2
            NOTE_DBC['Français']= frt

            #SES
            if isnan(df["Note à l'épreuve de Sciences économiques et sociales (SES)"][i]) == True:
                DBC['Validation']= 0
                DBC['COMMENTAIRE'] = "Aucune Notes de SES"
                return DBC, matiere
            else:
                ses = normalisation_bac(i,"Economique et social","Note à l'épreuve de Sciences économiques et sociales (SES)")
                NOTE_DBC['Sciences économiques et sociales']= ses

            #Spécialité pour le baccalauréat Scientifique, les notes de Mathématiques Spé , sont incluse avec la note de Math
            # On ne vas rechercher que les notes d'ISN
            SPE=df["Note à l'épreuve de Informatique et Sciences du numérique"][i]
            nsi = normalisation_bac(i,"Economique et social","Note à l'épreuve de Informatique et Sciences du numérique")
            if isnan(SPE) == False  :
                NSI = "OK"
                NOTE_DBC["Informatique et Sciences du numérique"]= nsi
            else:
                NSI = "NON"
            #Anglais
            LV1=df["LV1"][i]
            LV11=df["LV1.1"][i]
            LV12=df["LV1.2"][i]
            LV2=df["LV2"][i]
            LV21=df["LV2.1"][i]
            LV22=df["LV2.2"][i]
            Choix = ["Note à l'épreuve de Langue vivante 1","Note à l'épreuve de Langue vivante 2"]
            angl=None
            if((LV1=="Anglais" or LV11 =="Anglais") or LV12 =="Anglais"  ): 
                angl="Langue vivante 1"
                DBC["ANGLAIS"] = "LV1"
                agl = normalisation_bac(i,"Economique et social","Note à l'épreuve de Langue vivante 1")

            if((LV2=="Anglais" or LV21 =="Anglais") or LV22 =="Anglais"  ): 
                angl="Langue vivante 2"
                DBC["ANGLAIS"] = "LV2"
                agl = normalisation_bac(i,"Economique et social","Note à l'épreuve de Langue vivante 2")

            if(angl!=None) : 
                NOTE_DBC['Anglais']= agl 
            else:
                if NSI == "OK":
                    DBC['Validation']= 0
                    DBC['COMMENTAIRE'] = "Note de Français, MAthématiques et NSI OK. Aucune Note d'Anglais"
                else:
                    DBC['Validation']= 0
                    DBC['COMMENTAIRE'] = "Note de Français, MAthématiques OK. Aucune Note d'Anglais"
            return DBC , NOTE_DBC
    # Recherche des candidats STI2D        
    elif(Form=="Sciences et Technologies de l'Industrie et du Développement Durable") : 
        if MAT_FR(i) == "NON":
            DBC['Validation']= 0
            DBC['FORMATION'] = "Sciences et Technologies de l'Industrie et du Développement Durable"
            DBC['COMMENTAIRE'] = "Manque une note ou plus dans la matière 'Français'"
            return DBC , matiere
        else:
            
            DBC['Validation']=1
            DBC['FORMATION'] = "Sciences et Technologies de l'Industrie et du Développement Durable"

            #Math
            math = []
            if TRI_MANQUE(i,"Mathématiques") == False:
                DBC['Validation']= 0
                DBC['COMMENTAIRE'] = "Manque une notes ou plus en Mathématiques"
                return DBC, matiere
            else:
                math.append(moyenne_mat(df,i,"Mathématiques"))
                NOTE_STI2D['Mathématiques']= math[0]

            #Informatique
            if moyenne_mat(df,i,"Systemes d''information et numerique") == None:
                NSI = "NON"
            else:
                nsi = moyenne_mat(df,i,"Systemes d''information et numerique")
                NOTE_STI2D["Systemes d''information et numerique"]= nsi
                NSI = "OK"
            #Français
            fr = moyenne_mat(df,i,"Français")
            NOTE_STI2D['Français']=  fr   

            #Anglais
            LV1=df["LV1"][i]
            LV2=df["LV2"][i]
            mat=None
            if(LV1=="Anglais") : 
                mat="Langue vivante A"
                DBC["ANGLAIS"] = "LV1"
            if(LV2=="Anglais") : 
                mat="Langue vivante B"
                DBC["ANGLAIS"] = "LV2"

            if(mat!=None) : 
                angl = moyenne_mat(df,i,mat)
                NOTE_STI2D['Anglais']= angl
            else:
                if NSI == "OK":
                    DBC['Validation']= 0
                    DBC['COMMENTAIRE'] = "Note de Français, MAthématiques et Systemes d''information et numerique OK. Aucune Note d'Anglais"
                else:
                    DBC['Validation']= 0
                    DBC['COMMENTAIRE'] = "Note de Français, MAthématiques OK. Aucune Note d'Anglais"
            return DBC , NOTE_STI2D

    #Recherche des candidats de Série Générale (nouveau bac)
    elif(Form=='Série Générale'): 
        if MAT_FR(i) == "NON":
            DBC['Validation']= 0
            DBC['FORMATION'] = 'Série Générale'
            DBC['COMMENTAIRE'] = "Manque une note ou plus dans la matière 'Français'"
            return DBC , matiere
        else:
            DBC['Validation']=1
            DBC['FORMATION'] = "Série Générale"
            #Sciences
            if "Moyenne candidat en Enseignement scientifique Trimestre 1" in df.columns:
                matiere["NSI"] = moyenne_mat(df,i,"Enseignement scientifique")
            else:
                DBC['Validation']= 0
                DBC['COMMENTAIRE'] = "Manque une note ou plus en Enseignement Scientifique, et autres matières"
            #Français
            fr = []
            fr.append(moyenne_mat(df,i,"Français"))
            NOTE_SG['Français']= fr[0]

            #Anglais
            LV1=df["LV1"][i]
            LV2=df["LV2"][i]
            mat=None
            if(LV1=="Anglais") : 
                mat="Langue vivante A"
                DBC["ANGLAIS"] = "LV1"
            if(LV2=="Anglais") : 
                mat="Langue vivante B"
                DBC["ANGLAIS"] = "LV2"

            if(mat!=None) : 
                angl = moyenne_mat(df,i,mat)
                NOTE_SG['Anglais']= angl
            else:
                if NSI == "OK":
                    DBC['Validation']= 0
                    DBC['COMMENTAIRE'] = "Note de Français, MAthématiques et NSI OK. Aucune Note d'Anglais"
                else:
                    DBC['Validation']= 0
                    DBC['COMMENTAIRE'] = "Note de Français, MAthématiques OK. Aucune Note d'Anglais"
                return DBC , NOTE_DBC

            #Enseignements de spécialités
            mat1 = df["EDS BAC Terminale"][i]
            mat2 = df["EDS BAC Terminale.1"][i]
            if (mat1 == "Mathématiques Spécialité") or (mat2 == "Numérique et Sciences Informatiques"):
                if "Moyenne candidat en Mathématiques Spécialité Trimestre 1" in df.columns:
                    NOTE_SG["Mathématiques Spé"] = moyenne_mat(df,i,"Mathématiques Spécialité")
                if "Moyenne candidat en Numérique et Sciences Informatiques Trimestre 1" in df.columns:
                    NOTE_SG["Informatique et Sciences du numérique"] = moyenne_mat(df,i,"Numérique et Sciences Informatiques")
        #Si le candidats ne rentres dans aucune des conditions  
            return DBC , NOTE_SG
    else :
        DBC['Validation']=-1
        DBC['FORMATION'] = Form
        com = "Formation :'"+str(Form)+"' qui ne correspond pas aux critères"
        DBC['COMMENTAIRE'] = com
        return DBC , matiere
    

In [ ]:
NOTE_STI2D

In [281]:
ELIMINATION={}
FICHE_CANDIDATS_MANUEL={}
A_TRIER2={}
FICHE_STI2D={}
NOTE_STI2D = {}
FICHE_GENERAUX={}
NOTE_GENERAUX={}

for i in range(len(df)) :
    Form = df['Série diplôme'][i]
    groupe = df['Groupe'][i]

    FICHE , NOTES = recupDBC(df, i)
    num= df['ID'][i]
    
    if(FICHE["Validation"]==-1) : 
        ELIMINATION[num]=FICHE

        
    if(FICHE["Validation"]==0) : 
        FICHE_CANDIDATS_MANUEL[num]=FICHE

    if(FICHE["Validation"]==1 and groupe== 7415) : 
        FICHE_GENERAUX[num]=FICHE
        NOTE_GENERAUX[num]=NOTES

    if(FICHE["Validation"]==1 and groupe == 142309 ) : 
        FICHE_STI2D[num]=FICHE
        NOTE_STI2D[num]=NOTES
        
'''
AC = pd.DataFrame.from_dict(ELIMINATION)
ACT = AC.T
ACT.to_excel('ELIMINATION.xlsx', index=False)
AC = pd.DataFrame.from_dict(FICHE_CANDIDATS_MANUEL)
ACT = AC.T
ACT.to_excel('FICHE_CANDIDATS_MANUEL.xlsx', index=False)
AC = pd.DataFrame.from_dict(FICHE_GENERAUX)
ACT = AC.T
ACT.to_excel('FICHE_GENERAUX.xlsx', index=False)
AC = pd.DataFrame.from_dict(NOTE_GENERAUX)
ACT = AC.T
ACT.to_excel('NOTE_GENERAUX.xlsx', index=False)
AC = pd.DataFrame.from_dict(FICHE_GENERAUX)
ACT = AC.T
ACT.to_excel('FICHE_STI2D.xlsx', index=False)
AC = pd.DataFrame.from_dict(NOTE_GENERAUX)
ACT = AC.T   
ACT.to_excel('NOTE_SSTI2D.xlsx', index=False)
'''

"\nAC = pd.DataFrame.from_dict(ELIMINATION)\nACT = AC.T\nACT.to_excel('ELIMINATION.xlsx', index=False)\nAC = pd.DataFrame.from_dict(FICHE_CANDIDATS_MANUEL)\nACT = AC.T\nACT.to_excel('FICHE_CANDIDATS_MANUEL.xlsx', index=False)\nAC = pd.DataFrame.from_dict(FICHE_GENERAUX)\nACT = AC.T\nACT.to_excel('FICHE_GENERAUX.xlsx', index=False)\nAC = pd.DataFrame.from_dict(NOTE_GENERAUX)\nACT = AC.T\nACT.to_excel('NOTE_GENERAUX.xlsx', index=False)\nAC = pd.DataFrame.from_dict(FICHE_GENERAUX)\nACT = AC.T\nACT.to_excel('FICHE_STI2D.xlsx', index=False)\nAC = pd.DataFrame.from_dict(NOTE_GENERAUX)\nACT = AC.T   \nACT.to_excel('NOTE_SSTI2D.xlsx', index=False)\n"

In [ ]:
AC = pd.DataFrame.from_dict(A_CLASSE)
ACT = AC.T
ACT.to_excel('A_CLASSEE.xlsx', index=False)

In [ ]:
NG = pd.DataFrame.from_dict(NOTE_GENERAUX)
NGT = NG.T
NGT_NEW = NGT

In [ ]:
NGT

# CLASSEMENT DES GENERAUX

In [326]:
NGT.loc[NGT['Informatique et Sciences du numérique'] == 0, 'Informatique et Sciences du numérique'] = np.nan


In [327]:
# Définir une fonction pour normaliser les notes
def normalize_notes(column):
    x_min = column.min()
    x_max = column.max()
    return 20 / (x_max - x_min) * (column - x_min)

# Normaliser les notes sur une échelle de 0 à 20
for col in ['Mathématiques', 'Français', 'Informatique et Sciences du numérique', 'Anglais','Sciences économiques et sociales']:
    NGT[col] = normalize_notes(NGT[col])

In [328]:
coef_gen =[13,9,6,4,5]

In [329]:
def moyenne_cand(df,num,coef):
    som = 0
    divis = 0
    e = 0
    for mat in ['Mathématiques', 'Français', 'Informatique et Sciences du numérique', 'Anglais','Sciences économiques et sociales']:
        if isnan(df[mat][i]) == False:
            som += df[mat][i]*coef[e]
            divis += coef[e]
            e +=1
    res = som/divis
    return res
            

In [330]:
numro = NGT["Num_Can"].tolist()

In [331]:
for i in numro:
    NGT.at[i, "MOYENNE"] = moyenne_cand(NGT,i,coef_gen)

In [332]:
CLAS = NGT.sort_values('MOYENNE', ascending=False)

In [336]:
LISTE = CLAS["Num_Can"].tolist()
Classement_cand = {}

i = 1
while i <= len(LISTE):
    num = LISTE[i-1]
    Classement = {
        'Classement': i,
        'Num_Candidat': num,
        "Année du BAC": FICHE_GENERAUX[num]['ANNEE_BAC'],
        "Bourse": FICHE_GENERAUX[num]["Bourse"],
        "Moyenne":CLAS["MOYENNE"][num],
        "Mathématiques":NGT["Mathématiques"][num],
        "Français":NGT["Français"][num],
        "Systemes d''information et numerique":NGT['Informatique et Sciences du numérique'][num],
        'Sciences économiques et sociales':NGT['Sciences économiques et sociales'][num],
        "Anglais":NGT["Anglais"][num]
    }
    Classement_cand[int(LISTE[i-1])-1] = Classement  # affichage du classement courant
    i += 1


In [337]:
LECLASSEMENT = pd.DataFrame.from_dict(Classement_cand)
LECLASSEMENT = LECLASSEMENT.T
LECLASSEMENT.to_excel("classe.xlsx")

# Classement Candidats STI2D

In [311]:
ST = pd.DataFrame.from_dict(NOTE_STI2D)
STT = ST.T

In [312]:
# Définir une fonction pour normaliser les notes
def normalize_notes(column):
    x_min = column.min()
    x_max = column.max()
    if x_max != x_min:
        return 20 / (x_max - x_min) * (column - x_min)
    else:
        return None

# Normaliser les notes sur une échelle de 0 à 20
for col in ['Mathématiques', 'Français', "Systemes d''information et numerique", 'Anglais']:
    STT[col] = normalize_notes(STT[col])


In [315]:
coef_stid =[13,9,6,4]

In [316]:
def moyenne_cand(df,num,coef):
    som = 0
    divis = 0
    e = 0
    for mat in ['Mathématiques', 'Français', "Systemes d''information et numerique", 'Anglais']:
        if isnan(df[mat][i]) == False:
            som += df[mat][i]*coef[e]
            divis += coef[e]
            e +=1
    res = som/divis
    return res
            

In [317]:
numro = STT["Num_Can"].tolist()

In [318]:
for i in numro:
    STT.at[i, "MOYENNE"] = moyenne_cand(STT,i,coef_stid)

TypeError: must be real number, not NoneType

In [304]:
CLASSTD = STT.sort_values('MOYENNE', ascending=False)

KeyError: 'MOYENNE'

In [320]:
STT["Français"][3874]

19.405655052063796

In [322]:
LISTE = CLASSTD["Num_Can"].tolist()
Classement_cand = {}

i = 1
while i <= len(LISTE):
    num = LISTE[i-1]
    Classement = {
        'Classement': i,
        'Num_Candidat': num,
        "Année du BAC": FICHE_STI2D[num]['ANNEE_BAC'],
        "Bourse": FICHE_STI2D[num]["Bourse"],
        "Moyenne":CLASSTD["MOYENNE"][num],
        "Mathématiques":STT["Mathématiques"][num],
        "Français":STT["Français"][num],
        "Systemes d''information et numerique":STT["Systemes d''information et numerique"][num],
        "Anglais":STT["Anglais"][num]
        
    }
    Classement_cand[int(LISTE[i-1])-1] = Classement  # affichage du classement courant
    i += 1


In [323]:
FICHE_STI2D[4604]['ANNEE_BAC']

'2021-2022'

In [325]:
LECLASSEMENT = pd.DataFrame.from_dict(Classement_cand)
LECLASSEMENT = LECLASSEMENT.T
LECLASSEMENT.to_excel("classsthud.xlsx")